In [49]:
import tensorflow as tf
from tensorflow.contrib.learn import learn_runner
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import slim

mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [50]:
# Set tensorflow config
tf.set_random_seed(1)
tf.logging.set_verbosity(tf.logging.DEBUG)

In [3]:
# Define flags 
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string(
    flag_name='model_dir', 
    default_value='./estimator_model', 
    docstring='Output directory for model and training stats.'
)

In [51]:
def experiment_fn(run_config, params):
    
    run_config = run_config.replace(
        save_checkpoints_steps=params.min_eval_frequency)
    
    # Define the mnist classifier
    estimator = get_estimator(run_config, params)
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': mnist.train.images}, y=mnist.train.labels,
        batch_size=params.batch_size, num_epochs=None, shuffle=True)
    
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': mnist.test.images}, y=mnist.test.labels,
        batch_size=params.batch_size, shuffle=False)
    
    experiment = tf.contrib.learn.Experiment(
        estimator=estimator,  # Estimator
        train_input_fn=train_input_fn,  # First-class function
        eval_input_fn=test_input_fn,  # First-class function
        train_steps=params.train_steps,  # Minibatch steps
        min_eval_frequency=params.min_eval_frequency, # Hooks for evaluation
        eval_steps=None  # Use evaluation feeder until its empty
    )
    return experiment


In [52]:
# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    
    return out_layer

In [59]:
def model_fn(features, labels, mode, params): 
    """
    features(logits): All the feature vector. If the input_fn returns a dictionary of features, this will be a dictionary
    targets(labels): All the labels, same as features, but can be None if no labels are needed
    mode: ModeKeys. This is useful to decide how to build the model given the mode. If training you want to return a training operation, while in evaluation mode, you only need logits for example 
    params: a dictionary of params (Optional)
    """
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss_op,
        global_step=tf.contrib.framework.get_global_step(),
        optimizer=tf.train.AdamOptimizer,
        learning_rate=params.learning_rate
    )
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [60]:
def get_estimator(run_config, params):
    """Return the model as a Tensorflow Estimator object """
    return tf.estimator.Estimator(
        model_fn=model_fn, 
        params=params, 
        config=run_config
    )

In [61]:
def run_experiment(argv=None):
    ''' Run the training experiment '''
    
    # Define model parameters
    params = tf.contrib.training.HParams(
        batch_size = 128,
        learning_rate=0.002,
        n_classes=10,
        train_steps=5000,
        min_eval_frequency=100
    )

    # Set the run_config and the directory to save the model and stats
    run_config = tf.contrib.learn.RunConfig()
    run_config = run_config.replace(model_dir=FLAGS.model_dir)
    
    learn_runner.run(
        experiment_fn=experiment_fn,  # First-class function
        run_config=run_config,  # RunConfig
        schedule="train_and_evaluate",  # What to run
        hparams=params
    )

In [62]:
if __name__ == "__main__":
    tf.app.run(
        main=run_experiment
    )

INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_num_ps_replicas': 0, '_evaluation_master': '', '_save_checkpoints_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f75ecb9e8>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_environment': 'local', '_is_chief': True, '_keep_checkpoint_max': 5, '_session_config': None, '_log_step_count_steps': 100, '_master': '', '_num_worker_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './estimator_model', '_tf_random_seed': None, '_save_checkpoints_secs': None, '_task_id': 0, '_task_type': None}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Skipping training since max_steps has already saved.
Tensor("fifo_queue_DequeueUpTo:2", shape=(?,), dtype=uint8, device=/device:CPU:0)
INFO:tensorflow:Starting evaluation at 2017-10-11-05:24:51
INFO:tensorflow:Restoring parameters from ./estimator_model/

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
